In [ ]:
import sys
sys.path.insert(1, '../rtsvg')

from rtsvg import *

rt = RACETrack()

%matplotlib inline

In [ ]:
from math import cos, sin, pi

#
# Calculate the angled position string top and bottom position
#
def calculateAngledLabelTopAndBottomPosition(x, y, bar_w, txt_h, angle):
    frac_vert,frac_horz,bar_y = angle/90, (90-angle)/90, 0
    as_rad = pi*(angle+90)/180.0 # more than just radian conversion...
    horz_tpos  = (x+4,               y+4)       # top of string begin if the string were rendered horizontally
    horz_bpos  = (x+4,               y+4+txt_h) # bottom of string begin if the string were rendered horizontally
    vert_tpos  = (x+bar_w/2+txt_h/2, y+4)       # top of string begin if the string were rendered vertically
    vert_bpos  = (x+bar_w/2-txt_h/2, y+4)       # bottom of string begin if the string were rendered vertically
    angle_tpos = (vert_tpos[0]*frac_vert + horz_tpos[0]*frac_horz, vert_tpos[1]*frac_vert + horz_tpos[1]*frac_horz)
    angle_bpos = (angle_tpos[0] + cos(as_rad)*txt_h,               angle_tpos[1] + sin(as_rad)*txt_h)
    return angle_tpos,angle_bpos

#
# Does the specified angle cause the label to not overlap with the next label?
# ... there's a close formed solution here... but it's beyond me :(
# ... so many wasted cpu cycles... so many...
#
def doesAngleWorkForLabel(bar_w, txt_h, angle):
    if angle < 0 or angle >= 90:
        raise Exception(f'doesAngleWorkForLabel() - angle must be between [0,90) ... supplied angle = {angle}')

    # Position of label 0 and then label 1
    angle0_tpos,angle0_bpos = calculateAngledLabelTopAndBottomPosition(0,    0, bar_w, txt_h, angle)
    angle1_tpos,angle1_bpos = calculateAngledLabelTopAndBottomPosition(bar_w,0, bar_w, txt_h, angle)

    # Line from angle0_tpos in the direction of the angle...  is it underneath the angle1_bpos?
    m = sin(pi*angle/180)
    b = angle0_tpos[1] - m*angle0_tpos[0]
    return (m*angle1_bpos[0] + b) > angle1_bpos[1]

#
# Example Rendering
#
def createSVGExample(bar_w, txt_h, w=800, h=200):

    svg  = f'<svg width="{w}" height="{h}">'
    svg += f'<rect x="0" y="0" width="{w}" height="{h}" fill="#404040"/>'
    s   = 'AbCdEfGhIjKlMnOpQrStUvWxYz'

    for a in range(1,89):
        if doesAngleWorkForLabel(bar_w, txt_h, a):
            break

    as_rad = pi*(a+90)/180.0

    first_rendered, second_rendered = False, False
    last_pos,       last_last_pos   = None,  None

    bar_y = txt_h
    for x in range(bar_w,w-bar_w,bar_w):
        svg       += f'<line x1="{x+2}" y1="{bar_y}" x2="{x+bar_w-2}" y2="{bar_y}" stroke="#ffffff" />'

        horz_tpos = (x+4,               bar_y+4)       # top of string begin if the string were rendered horizontally
        horz_bpos = (x+4,               bar_y+4+txt_h) # bottom of string begin if the string were rendered horizontally
        vert_tpos = (x+bar_w/2+txt_h/2, bar_y+4)       # top of string begin if the string were rendered vertically
        vert_bpos = (x+bar_w/2-txt_h/2, bar_y+4)       # bottom of string begin if the string were rendered vertically
        
        svg       += f'<circle cx="{horz_tpos[0]}" cy="{horz_tpos[1]}" r="1" stroke="#ff0000" />'
        svg       += f'<circle cx="{vert_tpos[0]}" cy="{vert_tpos[1]}" r="1" stroke="#ffffff" />'
        svg       += f'<circle cx="{horz_bpos[0]}" cy="{horz_bpos[1]}" r="1" stroke="#ff0000" />'
        svg       += f'<circle cx="{vert_bpos[0]}" cy="{vert_bpos[1]}" r="1" stroke="#ffffff" />'

        last_last_pos = last_pos
        last_pos      = [horz_bpos,vert_bpos]

        frac_vert,frac_horz = a/90, (90-a)/90
        angle_tpos = (vert_tpos[0]*frac_vert + horz_tpos[0]*frac_horz, vert_tpos[1]*frac_vert + horz_tpos[1]*frac_horz)
        angle_bpos = (angle_tpos[0] + cos(as_rad)*txt_h, angle_tpos[1] + sin(as_rad)*txt_h)

        svg       += f'<circle cx="{angle_tpos[0]}" cy="{angle_tpos[1]}" r="1" stroke="#00ff00" />'
        svg       += f'<circle cx="{angle_bpos[0]}" cy="{angle_bpos[1]}" r="1" stroke="#00ff00" />'

        if   first_rendered  == False:
            svg += rt.svgText(s, angle_bpos[0], angle_bpos[1], txt_h, color="#E0E0E0", rotation=a)
            first_rendered = True
        elif second_rendered == False:
            svg += rt.svgText(s, angle_bpos[0], angle_bpos[1], txt_h, color="#E0E0E0", rotation=a)
            second_rendered = True

    svg += rt.svgText(s, last_pos[0][0],      last_pos[0][1],      txt_h, color="#E0E0E0")
    svg += rt.svgText(s, last_last_pos[1][0], last_last_pos[1][1], txt_h, color="#E0E0E0", rotation=90)

    return svg + '</svg>'


In [ ]:

rt.displaySVG(createSVGExample(32,24))

In [ ]:
rt.displaySVG(createSVGExample(64,24))

In [ ]:
rt.displaySVG(createSVGExample(32,12))

In [ ]:
rt.displaySVG(createSVGExample(16,12))